# Shortcut Models 公式（精简版）

Shortcut Models 在 Flow Matching 的基础上，给速度网络增加一个**步长条件** $d$，
使模型能用 1 步到多步灵活采样。

## 1) 回顾：Flow Matching 的问题

FM 学一个瞬时速度场 $v_\theta(x_t, t)$，采样时从噪声 $x_0$ 到图像 $x_1$ 需要沿着 ODE 轨迹一小步一小步走：

$$x_{t+\Delta t} = x_t + v_\theta(x_t, t) \cdot \Delta t$$

想象你在一条弯曲的山路上开车：
- **速度场** $v_\theta(x_t, t)$ 就像 GPS 告诉你"当前位置往哪个方向转方向盘"
- 你必须**一小段一小段开**（50-100步），因为路是弯的，走太大步就冲出弯道了
- 这就是为什么 FM 采样慢——每一步都要问一次 GPS（一次神经网络前向传播）

**能不能不走弯路，直接传送到终点？** 这就是 Shortcut Models 要解决的。

---

## 2) 什么是步长 $d$？

在 FM 中，时间 $t$ 从 0（纯噪声）走到 1（干净图像）。$d$ 就是你**一步想跨多远**。

| 步数 $K$ | 步长 $d = 1/K$ | 含义 |
|----------|---------------|------|
| 128 | $d = 1/128 \approx 0.008$ | 每步挪一点点，和标准 FM 一样精细 |
| 8 | $d = 1/8 = 0.125$ | 8步走完，比较快 |
| 2 | $d = 1/2 = 0.5$ | 两大步跨到终点 |
| 1 | $d = 1$ | **一步到位！** 从噪声直接跳到图像 |

用开车的比喻：
- $d$ 小（如 $1/128$）= GPS 说"往前开 10 米后再问我" → 很准但很慢
- $d$ 大（如 $1$）= GPS 说"直接传送到目的地" → 很快但你得知道精确方向

**Shortcut Models 的关键**：让模型**同时学会**所有步长下的最优速度。
给模型多一个输入 $d$，告诉它"我这一步要跨多远，请告诉我最优方向"：

$$s_\theta(x_t, t, d) \quad \leftarrow \text{多了一个 } d \text{ 输入}$$

---

## 3) Self-Consistency Loss：核心直觉

### 问题：大步的 ground truth 从哪来？

小步（$d \to 0$）的正确答案我们知道——就是 Flow Matching 的目标 $x_1 - x_0$。
但大步（$d = 1$）的正确答案**没有人告诉我们**。怎么办？

### 比喻：用短距离跳远教长距离跳远

想象你是跳远教练，要教运动员跳不同距离：

1. **1米**：你知道怎么跳（有 ground truth），直接教 ✅
2. **2米**：你不知道最佳姿势，但你知道——**跳2米应该等于连跳两次1米到达的位置** ✅
3. **4米**：你也不知道，但——**跳4米 = 连跳两次2米** ✅
4. **8米**：**跳8米 = 连跳两次4米** ✅
5. ...一直传递到任意距离！

这就是 **self-consistency bootstrap**：

$$\underbrace{d=0}_{\text{FM 给答案}} \xrightarrow{\text{一大步=两小步}} d=\frac{1}{128} \xrightarrow{} d=\frac{1}{64} \xrightarrow{} \cdots \xrightarrow{} d=1$$

### 公式

Self-Consistency 约束写成数学就是：

> **跳一大步 $2d$ 的速度** = **跳两小步 $d$ 的平均速度**

$$s_\theta(x_t, t, 2d) = \frac{s_{\theta'}(x_t, t, d) + s_{\theta'}(x_{t+d}, t+d, d)}{2}$$

其中 $x_{t+d} = x_t + d \cdot s_{\theta'}(x_t, t, d)$（先用 EMA 模型走一小步到中间点）。

为什么取**平均**？因为走 $2d$ 距离用速度 $v$ 只需一步：$\Delta x = 2d \cdot v$。
走两个 $d$ 步到同一位置：$\Delta x = d \cdot v_1 + d \cdot v_2 = d(v_1 + v_2)$。
令两者相等：$2d \cdot v = d(v_1 + v_2)$，所以 $v = (v_1 + v_2)/2$。

---

## 4) 两个训练损失

### Loss 1：Flow Matching Loss（教会小步）

$$\mathcal{L}_{\text{FM}} = \| s_\theta(x_t, t, 0) - (x_1 - x_0) \|^2$$

- 和标准 FM **完全一样**，只是固定 $d=0$
- 提供"1米跳远"的正确答案
- 占 batch 的 **75%**

### Loss 2：Self-Consistency Loss（从小步 bootstrap 到大步）

$$\mathcal{L}_{\text{SC}} = \left\| s_\theta(x_t, t, 2d) - \text{sg}\left(\frac{v_1 + v_2}{2}\right) \right\|^2$$

- $v_1 = s_{\theta'}(x_t, t, d)$：EMA 模型在当前位置的小步速度
- $v_2 = s_{\theta'}(x_{t+d}, t+d, d)$：EMA 模型在走一小步后位置的小步速度
- $\text{sg}(\cdot)$：stop-gradient，把目标当常数
- 教会"2米跳远 = 两次1米跳远"的道理
- 占 batch 的 **25%**

### Total Loss

$$\mathcal{L} = \mathcal{L}_{\text{FM}} + \mathcal{L}_{\text{SC}}$$

In [ ]:
# === Shortcut Models 训练的数值示例（无真实模型）===
import torch
import torch.nn.functional as F

B, C, H, W = 4, 3, 4, 4

# 模拟数据
x0 = torch.randn(B, C, H, W)  # 噪声
x1 = torch.randn(B, C, H, W)  # 数据

# === 采样步长 d ===
# d 从 {1/128, 1/64, 1/32, ..., 1} 中随机选一个
M = 128
j = torch.randint(0, 8, (B,))   # 指数 0..7
d = (2 ** j).float() / M         # d in {1/128, 1/64, ..., 1}

# t 在 d 的整数倍上采样，且 t + 2d <= 1
max_t_steps = (1.0 / d - 2).clamp(min=0).long()
t_steps = torch.stack([torch.randint(0, int(m)+1, (1,)) for m in max_t_steps]).squeeze()
t = t_steps.float() * d

t_ = t.view(B, 1, 1, 1)
d_ = d.view(B, 1, 1, 1)

# 线性插值
x_t = (1 - t_) * x0 + t_ * x1
v_true = x1 - x0  # ground truth velocity

print(f"d values: {d.tolist()}")
print(f"t values: {t.tolist()}")
print(f"x_t shape: {x_t.shape}")

In [ ]:
# === Loss 1: Flow Matching Loss ===
# 实际中：v_pred = s_theta(x_t, t, d=0)
v_pred_fm = v_true + 0.1 * torch.randn_like(v_true)  # 模拟预测

loss_fm = F.mse_loss(v_pred_fm, v_true)
print(f"Flow Matching Loss: {loss_fm.item():.4f}")

In [ ]:
# === Loss 2: Self-Consistency Loss ===

# 第一小步：EMA 模型在当前位置预测速度
v1 = v_true + 0.05 * torch.randn_like(v_true)  # 模拟 s_theta'(x_t, t, d)

# 用 EMA 模型走一小步到中间点
x_mid = x_t + d_ * v1  # x_{t+d}

# 第二小步：EMA 模型在中间点预测速度
v2 = v_true + 0.05 * torch.randn_like(v_true)  # 模拟 s_theta'(x_{t+d}, t+d, d)

# Bootstrap 目标：两小步速度的平均
v_target = (v1 + v2) / 2

# 模型预测大步速度
v_pred_big = v_true + 0.15 * torch.randn_like(v_true)  # 模拟 s_theta(x_t, t, 2d)

# 一大步 2d 的速度应该等于两小步 d 的平均速度
loss_sc = F.mse_loss(v_pred_big, v_target.detach())  # .detach() = stop-gradient
print(f"Self-Consistency Loss: {loss_sc.item():.4f}")

# Total
loss_total = loss_fm + loss_sc
print(f"Total Loss: {loss_total.item():.4f}")

---

## 5) 采样：同一个模型，任意步数

给定步数 $K$，设 $d = 1/K$：

$$x_{t+d} = x_t + d \cdot s_\theta(x_t, t, d)$$

从 $t=0$ 到 $t=1$ 走 $K$ 步。

**关键区别**：标准 FM 的模型没有 $d$ 输入，所以 $d$ 大了就不准。
Shortcut Model 因为训练时见过各种 $d$，所以大步也能走准。

In [ ]:
# === 采样对比 ===
import torch

def shortcut_sample(model_fn, num_steps):
    """Shortcut model: 模型知道步长 d"""
    x = torch.randn(1, 3, 4, 4)
    d = 1.0 / num_steps
    for i in range(num_steps):
        t = i * d
        v = model_fn(x, t, d)    # 多了 d！模型知道你要跨多远
        x = x + d * v
    return x

def fm_sample(model_fn, num_steps):
    """标准 FM: 模型不知道步长"""
    x = torch.randn(1, 3, 4, 4)
    dt = 1.0 / num_steps
    for i in range(num_steps):
        t = i * dt
        v = model_fn(x, t)       # 只有 x 和 t
        x = x + dt * v
    return x

fake_model = lambda x, t, d=None: -x * 0.5

print("=== Shortcut Models (同一个模型，不同步数) ===")
for K in [1, 2, 4, 8, 128]:
    torch.manual_seed(42)
    out = shortcut_sample(fake_model, K)
    print(f"  K={K:>3d} steps, d={1/K:.4f}, output norm: {out.norm().item():.4f}")

print()
print("=== Standard FM (步数少了就不准) ===")
for K in [1, 2, 4, 8, 128]:
    torch.manual_seed(42)
    out = fm_sample(fake_model, K)
    print(f"  K={K:>3d} steps, output norm: {out.norm().item():.4f}")

---

## 6) 总结

Shortcut Models 就干了两件事：

1. **给模型多一个输入 $d$**（你要跨多远），让模型能针对不同步长给出最优速度
2. **Self-Consistency Loss**（一大步 = 两小步），把小步的准确性 bootstrap 到大步

就这么简单。其他都和 Flow Matching 一样。